In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
import os
import datetime as dt
import sklearn

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_predict, cross_val_score

# logistic regression is our favourite model ever
from sklearn import linear_model

# used to calculate AUROC/accuracy
from sklearn import metrics

# local utils
import utils

import xgboost as xgb

import pickle

# default colours/marker/linestyles for prettier plots
col = [[0.9047, 0.1918, 0.1988],
    [0.2941, 0.5447, 0.7494],
    [0.3718, 0.7176, 0.3612],
    [1.0000, 0.5482, 0.1000],
    [0.4550, 0.4946, 0.4722],
    [0.6859, 0.4035, 0.2412],
    [0.9718, 0.5553, 0.7741],
    [0.5313, 0.3359, 0.6523]];
marker = ['v','o','d','^','s','o','+']
ls = ['-','-','-','-','-','s','--','--']

%matplotlib inline

/home/alistairewj/.virtualenvs/icu-model-transfer/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
df = pd.read_csv('X_mimic_day1.csv.gz', sep=',', index_col=0)

var_other = ['hospitalid', 'death', 'hosp_los', 'ventdays']
df_other = df[var_other]

# convenient reference to death column
y = df['death'].values
X = df.drop(var_other,axis=1)
print(X.columns)
X = X.values

print('{} observations. Outcome rate: {:2.2f}%.'.format(X.shape[0],
                                                        100.0*np.mean(y)))

Index(['heartrate_first', 'sysbp_first', 'diasbp_first', 'meanbp_first',
       'resprate_first', 'tempc_first', 'spo2_first', 'gcs_first',
       'bg_pao2_first_early', 'bg_paco2_first_early',
       'bg_pao2fio2ratio_first_early', 'bg_ph_first_early',
       'bg_baseexcess_first_early', 'albumin_first_early', 'bands_first_early',
       'bicarbonate_first_early', 'bilirubin_first_early', 'bun_first_early',
       'calcium_first_early', 'creatinine_first_early', 'glucose_first_early',
       'hematocrit_first_early', 'hemoglobin_first_early', 'inr_first_early',
       'lactate_first_early', 'platelet_first_early', 'potassium_first_early',
       'sodium_first_early', 'wbc_first_early', 'heartrate_last', 'sysbp_last',
       'diasbp_last', 'meanbp_last', 'resprate_last', 'tempc_last',
       'spo2_last', 'gcs_last', 'bg_pao2_last_early', 'bg_paco2_last_early',
       'bg_pao2fio2ratio_last_early', 'bg_ph_last_early',
       'bg_baseexcess_last_early', 'albumin_last_early', 'bands_last_

## Logistic Regression

In [3]:
np.random.seed(71017)

K = 5
idxK = np.random.permutation(X.shape[0])
idxK = idxK % K

# parameters from grid search
models = {'l2': linear_model.LogisticRegressionCV(n_jobs=4),
          'xgb': xgb.XGBClassifier(colsample_bytree=0.7, silent=1,
                            learning_rate=0.01, n_estimators=1000,
                            subsample=0.8, max_depth=9, n_jobs=4)
         }

mdl_val = dict()
results_val = dict()
smr_val = dict()
pred_val = dict()
pred_val_merged = dict()
for mdl in models:
    print('=============== {} ==============='.format(mdl))
    mdl_val[mdl] = list()
    results_val[mdl] = list() # initialize list for scores
    smr_val[mdl] = list()
    pred_val[mdl] = dict()
    pred_val_merged[mdl] = np.zeros(X.shape[0])
    
    if mdl == 'xgb':
        # no pre-processing of data necessary for xgb
        estimator = Pipeline([(mdl, models[mdl])])

    else:
        estimator = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("scaler", StandardScaler()),
                      (mdl, models[mdl])]) 

    for k in range(K):
        # train the model using all but the kth fold
        curr_mdl = sklearn.base.clone(estimator).fit(X[idxK != k, :], y[idxK != k])

        # get prediction on this dataset
        if mdl in ('lasso','ridge'):
            curr_prob = curr_mdl.predict(X[idxK == k, :])
        else:
            curr_prob = curr_mdl.predict_proba(X[idxK == k, :])
            curr_prob = curr_prob[:,1]
            
        pred_val_merged[mdl][idxK==k] = curr_prob
        pred_val[mdl][k] = curr_prob

        # calculate score (AUROC)
        curr_score = metrics.roc_auc_score(y[idxK == k], curr_prob)

        # add score to list of scores
        results_val[mdl].append(curr_score)
        
        # also get SMR
        smr_val[mdl].append(np.sum(y[idxK == k]) / np.sum(curr_prob))
        
        # save the current model
        mdl_val[mdl].append(curr_mdl)
        
        print('{} - Finished fold {} of {}. AUROC {:0.3f}.'.format(dt.datetime.now(), k+1, K, curr_score))
    
tar_val = dict()
for k in range(K):
    tar_val[k] = y[idxK==k]

=============== l2 ===============
2018-04-20 15:15:22.057881 - Finished fold 1 of 5. AUROC 0.883.
2018-04-20 15:15:24.376511 - Finished fold 2 of 5. AUROC 0.881.
2018-04-20 15:15:27.475063 - Finished fold 3 of 5. AUROC 0.876.
2018-04-20 15:15:30.330854 - Finished fold 4 of 5. AUROC 0.863.
2018-04-20 15:15:33.175742 - Finished fold 5 of 5. AUROC 0.873.
=============== xgb ===============
2018-04-20 15:16:14.841180 - Finished fold 1 of 5. AUROC 0.898.
2018-04-20 15:16:56.451509 - Finished fold 2 of 5. AUROC 0.906.
2018-04-20 15:17:57.020552 - Finished fold 3 of 5. AUROC 0.889.
2018-04-20 15:19:14.636023 - Finished fold 4 of 5. AUROC 0.884.
2018-04-20 15:20:35.878705 - Finished fold 5 of 5. AUROC 0.889.


## Save cross-validation performance to file

In [4]:
with open('model-performance-cv.csv', 'w') as fp:
    fp.write('{},{},{}\n'.format('model','auc','smr'))
    for mdl in results_val:
        fp.write('{}'.format(mdl))
        fp.write(',{}'.format(np.mean(results_val[mdl])))
        fp.write(',{}'.format(np.mean(smr_val[mdl])))
        fp.write('\n')